In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import scipy as sp
import os
%load_ext autoreload
%autoreload 2

# data path, constants, etc.
import sys
datapath = '/Users/nhamilto/Documents/ATom/coderepo/ATom/'
sys.path.append(datapath)

# plot things
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')

# Acoustic tomography functions
import atom_functions as atom
import TDSI as TDSI

In [8]:
datapath = '../../data/20180914_processed_data/'
ttfiles = [datapath+file for file in os.listdir(datapath) if 'tt' in file]

In [9]:
ttfiles

['../../data/20180914_processed_data/20180913150849_tt.csv',
 '../../data/20180914_processed_data/20180913150139_tt.csv',
 '../../data/20180914_processed_data/20180913144924_tt.csv',
 '../../data/20180914_processed_data/20180913151457_tt.csv',
 '../../data/20180914_processed_data/20180913150443_tt.csv',
 '../../data/20180914_processed_data/20180913145430_tt.csv',
 '../../data/20180914_processed_data/20180913145025_tt.csv',
 '../../data/20180914_processed_data/20180913151902_tt.csv',
 '../../data/20180914_processed_data/20180913145633_tt.csv',
 '../../data/20180914_processed_data/20180913145127_tt.csv',
 '../../data/20180914_processed_data/20180913150545_tt.csv',
 '../../data/20180914_processed_data/20180913145532_tt.csv',
 '../../data/20180914_processed_data/20180913150646_tt.csv',
 '../../data/20180914_processed_data/20180913145228_tt.csv',
 '../../data/20180914_processed_data/20180913151558_tt.csv',
 '../../data/20180914_processed_data/20180913151153_tt.csv',
 '../../data/20180914_pr

In [69]:
tmp = TDSI.meanField(ttfiles[0])

new mean field


In [72]:
tmp.path_lengths.flatten()

(64,)

In [74]:
path_velocity = pd.DataFrame()
for col in tmp.travel_times.columns:
    path_velocity[col] = tmp.travel_times[col]/tmp.path_lengths.flatten()

In [76]:
path_velocity.head()

,,,frame 0,frame 1,frame 2,frame 3,frame 4,frame 5,frame 6,frame 7,frame 8,frame 9,...,frame 110,frame 111,frame 112,frame 113,frame 114,frame 115,frame 116,frame 117,frame 118,frame 119
speaker,mic,pathID,,,,,,,,,,,,,,,,,,,,,
S0,M0,0,161.377213,161.370156,161.384269,161.391326,161.419552,161.433666,161.461892,161.483062,161.504231,161.532458,...,161.786495,161.814722,161.807665,161.800608,161.779439,160.657440,161.772382,161.779439,161.765325,161.765325
S1,M0,1,5.951980,5.951821,5.952139,5.952298,5.952936,5.953254,5.953891,5.954369,5.954847,5.955484,...,5.961218,5.961855,5.961696,5.961536,5.961059,5.935734,5.960899,5.961059,5.960740,5.960740
S2,M0,2,7.259572,7.259461,7.259683,7.259794,7.260237,7.260458,7.260901,7.261233,7.261566,7.262009,...,7.265996,7.266439,7.266328,7.266217,7.265885,7.248276,7.265774,7.265885,7.265663,7.265663
S3,M0,3,2.790994,2.790928,2.791059,2.791125,2.791388,2.791520,2.791783,2.791980,2.792177,2.792440,...,2.794807,2.795070,2.795004,2.794939,2.794741,2.784287,2.794676,2.794741,2.794610,2.794610
S4,M0,4,4.778448,4.778386,4.778510,4.778572,4.778820,4.778945,4.779193,4.779379,4.779566,4.779814,...,4.782050,4.782298,4.782236,4.782174,4.781988,4.772113,4.781926,4.781988,4.781864,4.781864


In [77]:
tmp.travel_times.head()

,,,frame 0,frame 1,frame 2,frame 3,frame 4,frame 5,frame 6,frame 7,frame 8,frame 9,...,frame 110,frame 111,frame 112,frame 113,frame 114,frame 115,frame 116,frame 117,frame 118,frame 119
speaker,mic,pathID,,,,,,,,,,,,,,,,,,,,,
S0,M0,0,114.345,114.34,114.35,114.355,114.375,114.385,114.405,114.42,114.435,114.455,...,114.635,114.655,114.65,114.645,114.63,113.835,114.625,114.63,114.62,114.62
S1,M0,1,186.845,186.84,186.85,186.855,186.875,186.885,186.905,186.92,186.935,186.955,...,187.135,187.155,187.15,187.145,187.13,186.335,187.125,187.13,187.12,187.12
S2,M0,2,327.745,327.74,327.75,327.755,327.775,327.785,327.805,327.82,327.835,327.855,...,328.035,328.055,328.05,328.045,328.03,327.235,328.025,328.03,328.02,328.02
S3,M0,3,212.245,212.24,212.25,212.255,212.275,212.285,212.305,212.32,212.335,212.355,...,212.535,212.555,212.55,212.545,212.53,211.735,212.525,212.53,212.52,212.52
S4,M0,4,384.695,384.69,384.70,384.705,384.725,384.735,384.755,384.77,384.785,384.805,...,384.985,385.005,385.00,384.995,384.98,384.185,384.975,384.98,384.97,384.97


In [81]:
tmp.travel_times['frame 0'].values.reshape(8,8).round(2)

array([[114.34, 186.84, 327.74, 212.24, 384.7 , 425.9 , 226.8 , 252.2 ],
       [206.14,  96.3 , 268.6 , 185.9 , 385.2 , 468.64, 294.85, 345.44],
       [247.58, 165.63, 197.53, 117.33, 325.48, 431.73, 279.93, 366.48],
       [340.65, 290.85, 320.85,  -3.85, 225.75, 374.4 , 264.45, 413.  ],
       [355.32, 332.07, 370.87,  63.37, 157.77, 313.32, 218.07, 394.82],
       [360.38, 378.98, 440.62, 174.78, 271.73, 199.88, 124.42, 340.92],
       [321.72, 365.72, 449.37, 225.02, 336.52, 279.82,  39.02, 266.77],
       [237.35, 306.9 , 426.7 , 264.25, 403.8 , 386.2 , 152.1 , 135.75]])